## Prepare Data

Get seasons wise kickers and pro bowl data

In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.mlab as mlab
from mysql.connector import errorcode
import matplotlib.pyplot as plt
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.html.widgets import *
import plotly as py
import plotly.graph_objs as go

%matplotlib inline

C:\Users\deeka\Anaconda3\envs\GTU\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [8]:
CONFIG = {
    'user': 'db_gtown_2018',
    'password': 'Gtown2018',
    'port': '3306',
    'host': 'nflnumbers.czuayagz62va.us-east-1.rds.amazonaws.com',
    'database': 'db_nfl',
    'raise_on_warnings': True,
}

QUERY = """SELECT 
        FKICKER,        
        Percentage.GID,
        GAME.SEAS AS SEASON,
        ((SUM(SUCCESS) / COUNT(*)) * 100) AS SUCCESS_PRCNTG,
        ((SUM(BLOCKED) / COUNT(*)) * 100) AS BLOCKED_PRCNTG,
        COUNT(*) AS PLAY_COUNT,
        CASE
            WHEN SUM(HIGH_PRESSURE) = 0 THEN 0
            ELSE ((SUM(HIGH_PRESSURE_SUCCESS) / SUM(HIGH_PRESSURE)) * 100)
        END AS HIGH_PRESSURE_SUCCESS_PRCNTG,
        SUM(HIGH_PRESSURE) AS HIGH_PRESSURE_COUNT,
        CASE
            WHEN SUM(LONG_DIST) = 0 THEN 0
            ELSE ((SUM(LONG_DIST_SUCCESS) / SUM(LONG_DIST)) * 100)
        END AS LONG_DIST_SUCCESS_PRCNTG, 
        SUM(LONG_DIST) AS LONG_DIST_COUNT,
        TEMP,
        HUMD,
        WSPD,
        (YEAR(CURDATE()) - PLAYER.start) AS YEARS_PLAYED,
        height,
        weight,
        PLAYER.forty,
        PLAYER.bench,
        PLAYER.vertical,
        PLAYER.broad,
        PLAYER.shuttle,
        PLAYER.cone,
        PLAYER.arm,
        PLAYER.hand,
        PLAYER.dcp,
        DPOS AS DRAFT_POSITION,
        (SELECT 
                (TO_DAYS(NOW()) - TO_DAYS(STR_TO_DATE(dob, '%m/%d/%Y'))) / 365
            FROM
                db_nfl.PLAYER P
            WHERE
                P.player = FKICKER) AS AGE_YEARS,
        CASE WHEN PRO_BOWL.ProBowl_Level IS NULL THEN 0 ELSE 1 END AS INPROBOWLTEAM
    FROM
        (SELECT 
            FGXP,
                PBP.GID,
                PBP.FKICKER,
                CASE
                    WHEN GOOD = 'Y' THEN 1
                    ELSE 0
                END AS SUCCESS,
                CASE
                    WHEN UPPER(DETAIL) LIKE '%BLOCKED%' THEN 1
                    ELSE 0
                END AS BLOCKED,
                CASE
                    WHEN
                        QTR IN ('2' , '4') AND MIN <= 2
                            AND (PTSO - PTSD) >= - 3
                    THEN
                        1
                    ELSE 0
                END AS HIGH_PRESSURE,
                CASE
                    WHEN
                        QTR IN ('2' , '4') AND MIN <= 2
                            AND (PTSO - PTSD) >= - 3
                            AND GOOD = 'Y'
                    THEN
                        1
                    ELSE 0
                END AS HIGH_PRESSURE_SUCCESS,
                CASE
                    WHEN DIST >= 50 THEN 1
                    ELSE 0
                END AS LONG_DIST,
                CASE
                    WHEN DIST >= 50 AND GOOD = 'Y' THEN 1
                    ELSE 0
                END AS LONG_DIST_SUCCESS,
                DIST - (100 - CASE
                    WHEN YFOG = '' THEN '98'
                    ELSE YFOG
                END) AS YDS_BEHIND_LOS
        FROM
            db_nfl.PBP
        LEFT OUTER JOIN db_nfl.PLAYER PLAYER ON PBP.FKICKER = PLAYER.PLAYER
        WHERE POS1 = 'K' ) AS Percentage
            LEFT OUTER JOIN
        db_nfl.PLAYER PLAYER ON Percentage.FKICKER = PLAYER.PLAYER
            LEFT OUTER JOIN
        db_nfl.GAME ON Percentage.GID = GAME.GID 
        LEFT OUTER JOIN
        db_nfl.PRO_BOWL ON PRO_BOWL.PLAYER_ID = PLAYER.PLAYER AND GAME.SEAS = ProBowl_Year        
    GROUP BY FKICKER , GAME.SEAS
    ORDER BY FKICKER , GAME.SEAS;"""

FEATURES = ['FKICKER','GID','SEASON','SUCCESS_PRCNTG','BLOCKED_PRCNTG','PLAY_COUNT',
        'HIGH_PRESSURE_SUCCESS_PRCNTG','HIGH_PRESSURE_COUNT','LONG_DIST_SUCCESS_PRCNTG','LONG_DIST_COUNT',
        'TEMP','HUMD','WSPD', 
        'YEARS_PLAYED','height','weight','forty','bench',
        'vertical','broad','shuttle','cone',
        'arm','hand','dcp','DRAFT_POSITION','AGE_YEARS','INPROBOWLTEAM']

def fetch_data():   
    try:
        cnx = mysql.connector.connect(**CONFIG)
        cursor = cnx.cursor()
        #Let's read all the rows in the table
        cursor.execute(QUERY)
        #specify the attributes that you want to display
        df = DataFrame(cursor.fetchall())    
        df.columns = FEATURES
        cnx.commit()
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    else:
        
        return df
        cursor.close()
        cnx.close()
DATA = fetch_data()     

In [9]:
DATA.head(3)

,FKICKER,GID,SEASON,SUCCESS_PRCNTG,BLOCKED_PRCNTG,PLAY_COUNT,HIGH_PRESSURE_SUCCESS_PRCNTG,HIGH_PRESSURE_COUNT,LONG_DIST_SUCCESS_PRCNTG,LONG_DIST_COUNT,...,vertical,broad,shuttle,cone,arm,hand,dcp,DRAFT_POSITION,AGE_YEARS,INPROBOWLTEAM
0,AD-0800,14,2000,86.8421,2.6316,76,73.6842,19,100.0000,1,...,0.0,0,0.0,0.0,0,0.0,0,0,56.2658,0
1,AE-0700,793,2003,89.1892,0.0000,74,94.4444,18,33.3333,3,...,0.0,0,0.0,0.0,0,0.0,0,0,40.0082,0
2,AE-0700,1054,2004,75.0000,0.0000,4,100.0000,1,0,0,...,0.0,0,0.0,0.0,0,0.0,0,0,40.0082,0


In [60]:
DATA[['GID','SEASON','PLAY_COUNT','SUCCESS_PRCNTG','BLOCKED_PRCNTG','HIGH_PRESSURE_SUCCESS_PRCNTG','HIGH_PRESSURE_COUNT','LONG_DIST_SUCCESS_PRCNTG','LONG_DIST_COUNT','INPROBOWLTEAM']]=DATA[['GID','SEASON','PLAY_COUNT','SUCCESS_PRCNTG','BLOCKED_PRCNTG','HIGH_PRESSURE_SUCCESS_PRCNTG','HIGH_PRESSURE_COUNT','LONG_DIST_SUCCESS_PRCNTG','LONG_DIST_COUNT','INPROBOWLTEAM']].apply(pd.to_numeric)

In [62]:
import time 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split as tts 

# Identify our data and target 
X = DATA[['GID','SEASON','PLAY_COUNT','SUCCESS_PRCNTG','BLOCKED_PRCNTG','HIGH_PRESSURE_SUCCESS_PRCNTG','HIGH_PRESSURE_COUNT','LONG_DIST_SUCCESS_PRCNTG','LONG_DIST_COUNT']]
y = DATA['INPROBOWLTEAM']

# Create random train and test splits
splits = tts(X, y, test_size=0.2)
X_train, X_test, y_train, y_test = splits

In [63]:
def fit_model(model, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    start = time.time()
    model.fit(X_train, y_train)
    duration = time.time() - start 
    score = model.score(X_test, y_test)
    
    print("{} fit in {:0.2f} seconds score: {:0.4f}".format(model.__class__.__name__, duration, score))
    print(model.get_params())

In [64]:
models = (
    LogisticRegression(), 
    SVC(),
    RandomForestClassifier(),
    Perceptron(), 
    KNeighborsClassifier(),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=2),
    GaussianNB(), 
    MultinomialNB(),
)

for model in models:
    fit_model(model)

LogisticRegression fit in 0.01 seconds score: 0.9510
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 100, 'multi_class': 'ovr', 'n_jobs': 1, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
SVC fit in 0.03 seconds score: 0.9580
{'C': 1.0, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
RandomForestClassifier fit in 0.01 seconds score: 0.9650
{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': 1, 'oob_score': False, 'random_state'

C:\Users\deeka\Anaconda3\envs\GTU\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning:

max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.

